In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv("data_3_indexes.csv")
dataset = df.drop(["Date"], axis = 1)

In [6]:
# Viewing the dataset
dataset.head()

,HS Adj Close Diff,HS high-low % change,HS open-close % change,NIKKEI Adj Close Diff,NIKKEI high-low % change,NIKKEI open-close % change,SSE Adj Close Diff,SSE high-low % change,SSE open-close % change,SP500 Adj Close Diff,SP500 high-low % change,SP500 open-close % change
0,-0.937184,2.179445,-1.891697,-0.372786,0.857677,-0.579634,-4.209718,3.890297,-3.687441,1.405788,1.525389,1.425832
1,0.239589,1.898326,0.092961,-1.333813,1.288838,-1.137477,-5.700400,7.173637,-5.585579,-0.517427,1.491718,-0.514764
2,-0.448881,1.293047,-0.369097,0.749076,0.987323,0.617880,1.157740,4.632927,1.543474,0.712924,0.789856,0.718043
3,-0.601889,1.857523,-0.786756,-0.795651,2.399229,-1.122047,0.997013,1.683555,0.646524,-1.235210,2.166258,-1.220138
4,0.913122,2.060494,1.175447,0.291622,1.080233,0.228363,-0.061321,2.031630,-0.098548,0.682883,1.352410,0.687579


In [7]:
# Summary statistic of the dataset
dataset.describe()

,HS Adj Close Diff,HS high-low % change,HS open-close % change,NIKKEI Adj Close Diff,NIKKEI high-low % change,NIKKEI open-close % change,SSE Adj Close Diff,SSE high-low % change,SSE open-close % change,SP500 Adj Close Diff,SP500 high-low % change,SP500 open-close % change
count,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000,5097.000000
mean,-0.020254,1.511082,-0.044913,-0.014324,1.447936,-0.029786,0.007871,1.779601,0.074562,0.008581,1.363278,0.014619
std,1.595319,1.052522,1.166636,1.523485,1.056134,1.179449,1.557500,1.231619,1.417306,1.252157,1.045826,1.152388
min,-15.875448,0.000000,-10.966925,-12.874930,0.000000,-10.024620,-9.697993,0.062160,-9.086704,-13.615778,0.145747,-8.723100
25%,-0.729548,0.864097,-0.593222,-0.758386,0.818102,-0.577235,-0.671995,0.978662,-0.591614,-0.491836,0.711193,-0.478899
50%,0.037409,1.229924,-0.029762,0.028823,1.203203,-0.022677,0.052146,1.423471,0.075548,0.066901,1.084164,0.056389
75%,0.759160,1.828611,0.519845,0.805136,1.775394,0.556880,0.739959,2.155704,0.726686,0.593180,1.677023,0.549201
max,15.841634,19.300333,12.924924,12.396210,26.106936,12.364858,8.972613,11.229284,8.180039,9.738336,11.520845,10.789006


In [8]:
from sklearn.preprocessing import MinMaxScaler
# prepare data for lstm
 
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
dataset = dataset
values = dataset.values

# ensure all data is float
values = values.astype('float32')

# frame as supervised learning
reframed = series_to_supervised(values, 1, 1)
reframed.drop(reframed.columns[[9,10,12,13,14,15,16,17,18,19,20,21,22]], axis=1, inplace=True)
print(reframed.head())
values = reframed.values


   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1  -0.937184   2.179445  -1.891697  -0.372786   0.857677  -0.579634   
2   0.239589   1.898326   0.092961  -1.333813   1.288838  -1.137477   
3  -0.448881   1.293047  -0.369097   0.749076   0.987323   0.617880   
4  -0.601889   1.857523  -0.786756  -0.795651   2.399229  -1.122047   
5   0.913122   2.060494   1.175447   0.291622   1.080233   0.228363   

   var7(t-1)  var8(t-1)  var9(t-1)  var12(t-1)  var12(t)  
1  -4.209718   3.890297  -3.687441    1.425833 -0.514764  
2  -5.700400   7.173637  -5.585579   -0.514764  0.718043  
3   1.157740   4.632927   1.543474    0.718043 -1.220138  
4   0.997013   1.683555   0.646524   -1.220138  0.687579  
5  -0.061321   2.031630  -0.098548    0.687579  0.317359  


In [9]:
# split into train and test sets
train_size = round(0.8 * len(values))
train = values[:train_size, :]
test = values[train_size:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# # reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(4077, 1, 10) (4077,) (1019, 1, 10) (1019,)


In [1]:
import sys
!conda install --yes --prefix {sys.prefix} keras

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                               failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - keras -> python[version='>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|>=3.6,<3.7.0a0']

Your python: python=3.8

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that co

In [2]:
import sys
!{sys.executable} -m pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)


In [10]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Bidirectional
from keras.layers import Dropout

# design network
model = Sequential()
model.add(LSTM(128, return_sequences= True, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(32)))
model.add(BatchNormalization())
model.add(Dense(16))
model.add(Dense(1))
model.compile(loss='Huber', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(test_X, test_y), verbose=2, shuffle=True)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'keras'

In [ ]:
import math
# make a prediction
yhat = model.predict(train_X)
# calculate RMSE
rmse = math.sqrt(mean_squared_error(train_y, yhat))
print('Train RMSE: %.3f' % rmse)

plt.figure(figsize=(20,5))
plt.plot(yhat)
plt.plot(train_y,alpha=0.5)
plt.show()

In [ ]:
import math
# make a prediction
yhat = model.predict(test_X)

# calculate RMSE
rmse = math.sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.3f' % rmse)

plt.figure(figsize=(20,5))
plt.plot(yhat)
plt.plot(test_y,alpha=0.6)
plt.show()